一般問題 資料建立

In [22]:
import os
import json
from units import get_time_text

qa_index = 1
only_for_classification = False

def make_dataset(only_for_classification):
    
    _, build_dataset_time = get_time_text()
    time_number = build_dataset_time.replace('-','').replace(' ','')

    dataset = {}
    dataset['name'] = 'test_dataset'
    dataset['build_dataset_time'] = build_dataset_time
    dataset['build_dataset_time_number'] = time_number
    dataset['only_for_classification'] = only_for_classification
    dataset['data_list'] = []
    dataset['token'] = 0
    
    score_data = {}
    score_data['Classifier'] = 0
    score_data['General'] = 0
    score_data['SQL'] = 0
    score_data['Retrieve'] = 0
    
    dataset['score_data'] = score_data
    dataset['Classifier Accuracy'] = 0
    
    return dataset
    
test_set = make_dataset(only_for_classification)
test_dataset_path = 'data/Owen_test.json' 

every_class_test_maximum = 70


查詢資料 建立

In [23]:
questions = [
    # ('B760M-K-CSM多少錢呀？', 'int', '10590'),
    # ('請幫我查詢H610M-E的品牌？', 'str', '微星'),
    # ('我想問B760M-D2H的價格多少，想詢問價格', 'int', '3090'),
    # ('H770-PRO的品質是哪一家？可以幫我確認嗎？', 'str', '華碩'),
    ('Intel Processor 300多少錢呀？', 'int', '2990'),
    ('詢價Intel i3-14100F價格？', 'int', '3990'),
    ('查Intel i3-14100價格？', 'int', '4800'),
    ('請幫我查Intel i5-14400F多少錢呀？', 'int', '6800'),
    ('幫查Intel i5-14600K價格？', 'int', '10150'),
    ('想查價Intel i7-14700K？', 'int', '13800'),
    ('查Intel i5-14600KF價格？', 'int', '9600'),
    ('想查Intel i7-14700價格是多少？', 'int', '12900'),
    ('想查Inteli5-14600KF價格？', 'int', '9600'),
    ('想查Intel i5-14400價格？', 'int', '7600'),
    
    # 以下為GPU
     ('詢價GT710-SL-2GD3-BRK-EVO？', 'int', '1690'),
     ('請查詢華碩GT710-SL-2GD3-BRK-EVO 的價錢？', 'int', '1690'),
     ('詢問GT710 1GD3H LP 的價錢？', 'int', '1450'),
     ('請查技嘉N730D5-2GL 的價錢？', 'int', '2690'),
     ('查GT1030 2GD4 LP OC 的價錢？', 'int', '2850'),
     ('查詢有一台微星GT1030GPU價錢為3290是哪一台？', 'string', 'GT1030 AERO ITX 2GD4 OCV1'),
     ('查有一台微星RTX3050??價格為6890是哪一台？', 'string', 'RTX3050 AERO ITX 8G OCV1'),

    #ram
###查價
     ('詢價UMAX 單條8GB DDR5-5600/CL46多少錢？', 'int', '749'),
     ('請查詢UMAX 單條16GB DDR5-5600/CL46的價錢？', 'int', '1299'),
     ('詢問UMAX 單條32GB DDR5-5600/CL46的價錢？', 'int', '2599	'),
     ('請查金士頓 單條16GB DDR5-5600 FURY Beast的價錢？', 'int', '1580'),
     ('金士頓 單條16GB DDR5-5600 FURY Beast RGB的價錢？', 'int', '1750'),
#     ('查詢有一台美光價錢為1399的全名？', 'string', '美光 Micron Crucial 單條16GB DDR5 5600/CL46'),
#     ('查有一台微星RTX3050??價格為6890是哪一台？', 'string', 'RTX3050 AERO ITX 8G OCV1'),
     ('想查美光Micron Crucial 單條32GB DDR5 5600/CL46價格？', 'int', '2799'),
     ('想查威剛 32GB(雙通16GB*2) DDR5 5600 XPG Lancer/CL36 黑	價格？', 'int', '3199'),
     ('想查KLEVV(科賦) 32GB(雙通道16GB*2) DDR5 6000/CL30 BOLT V價格？', 'int', '3299') ,
    ('目前金士頓最便宜的Ram是多少錢？', 'int', '3299'),
###測試性能
    ('目前Ram容量最大的是幾G？', 'int', '128') ,
    ('目前Ram容量最小的是幾G?','int','620'),
    

    #hdd
     ('詢價UMAX S330 240GB價格？', 'int', '490'),
     ('請查詢UMAX S330 480GB的價錢？', 'int', '880'),
     ('詢問威剛 Ultimate SU650 120G的價錢？', 'int', '350'),
     ('請查威剛 Ultimate SU650 240G的價錢？', 'int', '599'),
     ('WD Blue SA510 250GB的價錢？', 'int', '1100'),
     ('想查WD Blue SA510 500GB價格？', 'int', '1500'),
     ('想查金士頓 A400 240G價格？', 'int', '699'),
     ('想查十銓 T-Force Vulcan Z價格？', 'int', '699') ,
     ('查詢有一台美光價錢為630的產品名稱？', 'string', '美光 Crucial BX500 240G'),
     ('查有一台三星價格為1529是哪一台？', 'string', '三星 Samsung 870 EVO 500G'),
     ('目前SDD容量最高價錢多少？', 'int', '17999'),
     ('幫我查詢HDD容量最高，價錢最低的是哪個型號？', 'string', 'Seagate 22TB【那嘶狼 PRO】512M/7200轉/五年/3年 Rescue(ST22000NT001) -> Seagate 22TB【那嘶狼 PRO】'),
###測試性能
     ('目前HDD容量最大是多少G？', 'int', '22000'),
     ('目前HDD容量最大是多少TB？', 'int', '22'),
    ('目前HDD容量最高是哪個品牌？', 'string', 'Seagate'),
     ('目前SDD容量最高是哪個品牌？', 'string', '三星 Samsung'),
    
    #cool
     ('詢價利民 M.2 2280 TYPE A B SSD 固態硬碟散熱片/鋁合金/單雙面皆適用價格？', 'int', '299'),
     ('請查詢快睿 Frostbit M.2 散熱片/2280/雙導管/可調式散熱鰭片的價錢？', 'int', '790'),
     ('詢問Montech Air Cooler 210 6導管的價錢？', 'int', '1090'),
     ('請查ID-COOLING IS-40X 散熱器 下吹式的價錢？', 'int', '790'),
     ('be quiet! PURE ROCK 2 BLACK 4導管的價錢？', 'int', '1290'),
     ('想查COUGAR FORZA 50 ESSENTIAL價格？', 'int', '699'),
     ('想查利民 AXP90-X47 白化版 下吹式 4導管(6mm)價格？', 'int', '1050'),
     ('想查利民 Assassin X 120 R SE /4導管(6mm)價格？', 'int', '690') ,
#     ('查詢利民$350的產品名稱？', 'string', '利民 M.2 2280 SSD 固態硬碟散熱片'),
#     ('查有一台Montech價格為1590是哪一台？', 'string', 'Montech Metal DT24 Base 6導管'),
    
    
]

for question in questions[:every_class_test_maximum]:
    
    data = {}
    data['qa_index'] = str(qa_index)
    qa_index += 1
    data['class'] = '2'
    data['question'] = question[0]
    data['answer'] = question[2]
    data['answer_type'] = question[1]
    data['only_for_classification'] = only_for_classification
    test_set['data_list'].append(data)

推薦 資料建立

In [24]:
with open(test_dataset_path, 'w') as file:
    json.dump(test_set, file, ensure_ascii=False, indent=4)

In [25]:
### 驗證查詢
import sqlite3
import pandas as pd
db_name = '/home/pc_diy/crawler/data/sqlchain.db'
# db_name = '../crawler/data/cpu_and_gpu.db'
conn = sqlite3.connect(db_name)  
sql1=f'''
select *
from cpu
'''
test=pd.read_sql(sql1,conn)
conn.close()
test.head(50)


,NAME,PRICE,BRAND,ETL_DATE,performance,base_clock,boost_clock
0,Intel Processor 300,2990,Intel,2024-03-17,2核/4緒,3.9,3.9
1,Intel i3-14100F,3990,Intel,2024-03-17,4核/8緒,3.5,4.7
2,Intel i3-14100,4800,Intel,2024-03-17,4核/8緒,3.5,4.7
3,Intel i5-14400F,6800,Intel,2024-03-17,10核/16緒,2.5,4.7
4,Intel i5-14400,7600,Intel,2024-03-17,10核/16緒,2.5,4.7
5,Intel i5-14500,7800,Intel,2024-03-17,14核/20緒,2.6,5.0
6,Intel i5-14600KF,9600,Intel,2024-03-17,14核/20緒,3.5,5.3
7,Intel i5-14600K,10150,Intel,2024-03-17,14核/20緒,3.5,5.3
8,Intel i7-14700F,12000,Intel,2024-03-17,20核/28緒,2.1,5.4
9,Intel i7-14700,12900,Intel,2024-03-17,20核/28緒,2.1,5.4


In [26]:
### 驗證查詢
import sqlite3
import pandas as pd
db_name = '/home/pc_diy/crawler/data/sqlchain.db'
# db_name = '../crawler/data/cpu_and_gpu.db'
conn = sqlite3.connect(db_name)  
sql1=f'''
SELECT name FROM sqlite_master WHERE type='table';
'''
test=pd.read_sql(sql1,conn)
conn.close()
test.head(50)

,name
0,mainboard
1,ram
2,hdd
3,cpu
4,gpu
5,cool
6,motherboard
7,radiator
8,chassis
9,battery
